<p align="center">
  <img src="images/codete_workshops.jpg" />
</p>

### Preparation

Before you start, please read the preparation instructions.

### Short theory first

Topics that are related to bots and AI are the following:
* natural language processing,
* natural language understanding,
* sentiment analysis,
* context management,
* pattern recognition.

We focus in this workshop on NLU and sentiment analysis.

There are planty of bot types. One of the taxonomy is presented in [1] and looks like following:
* personal bots/team bots,
* voice/text bots,
* super bots and domain-specific bots,
* many others.

### Let's meet Anna our HR assistant

<p align="center">
  <img src="images/hr_bot_logo_S.png" />
</p>

She helps our HR department with the recruitment process. The main goals are:
1. Add, delete and move trello cards where all candidates are placed.
2. React on slack #hr channel, measure the sentiment or each message.

**Scenario 1**
1. Connect to Slack.
2. Send and receive messages from given channel.
3. Add sentiment analysis and analyse messages.

**Scenario 2**
1. Build a NLU model based on rasa package.
2. Based on received messages check the intent.
3. If the goal is to add a new candidate to trello, the bot should add a new card in Trello.

**Scenario 3**
1. Use Google Translate to imitate our voice.
2. Connect to AVS and get a response.
3. Build a "skill" that will get the current recruitment status from Trello and response (imitate) with a voice answer.

### Scenario 1

### Slack

You need to create an account first. It's good to create a new slack workspace, but you can use also a slack account where you have the admin priviliges as described in the preparation document.


In [1]:
SLACK_TOKEN = ""

#### Testing your Slack environment

In [ ]:
from slackclient import SlackClient

sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
     sc.api_call("chat.postMessage", channel="#general",
                          text="test", as_user=False) 
else:
    print("Connection failed")

In [ ]:
sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
     sc.api_call("chat.postMessage", channel="#general",
                          text="Hello. I'm Anna, your HR assistant. How can I help?", as_user=True)
else:
    print("Connection failed")

#### TASK 1: Identify with who the bot is getting the message from and get channel id by name.

Implement a util function ```get_user()```. As parameter we get an instance of ```SlackClient```. Use it to get the user lists by ```id``` as a map ```{'id':'name'}```. Find the proper method in the Slack API documentation: [2].

In [ ]:
def get_user_map(sc):
    users = {}
    # put your code here
    return users

Implement a util function ```get_channel_id_by_name()```. As parameter we get an instance of ```SlackClient```. Find the proper method in the Slack API documentation: [2].

In [ ]:
def get_channel_id_by_name(sc, channel_name):
    channel_id = ""
    # put your code here
    return channel_id

#### Read messages

In [ ]:
import time

sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
      #channel_id = get_channel_id_by_name(sc, "general")
      last_message_id = 0
      while True:
           response = sc.api_call("channels.history", channel="C76LMNLFP", count=1,latest="")
           if response['ok']:
                msg = response['messages'][0]
                if msg['ts'] != last_message_id:
                     if 'user' in msg:
                          #put your code here
                          print(msg["user"]+": "+ msg["text"])
                          last_message_id = msg['ts']
           time.sleep(5)       
else:
    print("Connection failed") 

### Sentiment analysis

At Codete Research Lab we have develop a solution for semantic analysis. More details about it can be found: [3] and [4].

In [ ]:
import requests
import json

def get_sentiment(sentence):
    url = "http://rest-ml-model-dev.eu-west-1.elasticbeanstalk.com/model/c8f8f825-1eff-4f5d-80d6-b6c65e81/prediction"
    payload = {"sentence":sentence}
    headers = {"Content-type": "application/json", "Accept": "application/json"}
    response = requests.put(url,json=payload,headers=headers)
    return response.json()

In [ ]:
import matplotlib.pyplot as plt

sentiment = get_sentiment("good Python skills")
#he had some good ideas for solutions
print(sentiment)
labels = ['Negative','Neutral','Positive']
colors = ['red','gray','green']
patches, texts = plt.pie(list(sentiment.values()),colors=colors,startangle=0)
plt.legend(patches, labels)
plt.axis('equal')
plt.tight_layout()
plt.show()

#### TASK 2: Combine sentiment analysis with Slack messages

Take the methods above and combine it together with slack messages that you retrieve from the users. If you have run methods like ```get_sentiment()```, you don't need to rewrite it again here. 

In [ ]:
sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
      last_message_id = 0
      while True:
            pass
        # your code goes here
else:
    print("Connection failed")     

### Scenario 2

Before we start here, we need to get the tokens to Trello. To obtain token we need to follow the steps:
1. Signup at http://trello.com and follow the registration instructions.
2. Go to https://trello.com/app-key to obtain the api-key, secret key and token. The API-KEY is available under Key on the top of the page. The API_SECRET_KEY is available on the bottom of the page under Secret. The TOKEN can be found under the Token link in the top of the page.

In [ ]:
API_KEY = ""
API_SECRET_KEY = ""
TOKEN = ""
TOKEN_SECRET = ""

In [ ]:
from trello import TrelloClient

client = TrelloClient(
    api_key=API_KEY,
    api_secret=API_SECRET_KEY,
    token=TOKEN,
    token_secret=TOKEN_SECRET
)

boards = client.list_boards()
print(boards)

#### Setup our board lists

If we have a clean setup of Trello, we probably have just one board, so we need to create our _Recruitment_ board and lists like: _Prescreening_, _Before interview_, _After interview_, _Finished_. It can be easily done by using Python Trello API. You can read more about the Trello Python library: [4]

In [ ]:
def setup_trello_board():
    recruitment_board = None
    for board in boards:
        if board.name == "Recruitment":
            recruitment_board= board
    if recruitment_board == None:
        recruitment_board = client.add_board("Recruitment")
    lists_to_create = ['Prescreening','Before interview','After interview','Finished']
    
    #trello_lists = recruitment_board.all_list()
    
    finished_list     = recruitment_board.add_list(lists_to_create[3])
    after_list        = recruitment_board.add_list(lists_to_create[2])
    before_list       = recruitment_board.add_list(lists_to_create[1])
    prescreening_list = recruitment_board.add_list(lists_to_create[0])        
    
    return recruitment_board, prescreening_list, before_list, after_list, finished_list

# run once
board, prescreening_list, before_list, after_list, finished_list = setup_trello_board()

#### Add a new card in trello

In [ ]:
prescreening_list.add_card("John Kowalski - Data Scientist","Some description goes here")

#### NLU

We use rasa library for the NLU. To start the server we need to execute it through Bash shell of our Docker container like following:

```python3 -m rasa_nlu.server &```

Most important part is the learning part as rasa does not know nothing about recruitment. You can learn more about this library by reading the documentation: [5]. It use some NLP libraries like [6]. It use a neural network. MITIE uses a SVM.

To train the NLU server, we should prepare a data set of intents, synonomis and entities, but first let's check how does it work and execute some examples on the server.

In [ ]:
def get_intent(sentence):
    url = "http://localhost:5000/parse"
    payload = {"q":sentence}
    response = requests.get(url,params=payload)    
    print(response.json())
    intent = response.json()['intent']
    if intent['confidence'] > 0.5: 
        return intent['name']
    return response.json()

get_intent("hi")

In [ ]:
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer

training_data = load_data('data.json')
trainer = Trainer(RasaNLUConfig("config.json"))
trainer.train(training_data)
model_directory = trainer.persist('.')

In [ ]:
from rasa_nlu.model import Metadata, Interpreter

interpreter = Interpreter.load(model_directory, RasaNLUConfig("config.json"))

interpreter.parse(u"add candidate")

#### TASK 3: Set a list of questions about the candidate if candidate_add intent appears

The bot should check the received messages and check using ```interpreter.parse()``` what kind of intent it is dealing with. If it's candidate_add intentent, Anna should ask questions for: forname, surname and position. All responses should be saved to trello in one card. If a response isn't clear Anna should ask for clarification.

In [ ]:
sc = SlackClient(slack_token)

if sc.rtm_connect():
      last_message_id = 0
      while True:
            pass            
        # your code goes here
else:
    print("Connection failed")

#### TASK 4: Create a new training data set for NLU for candidate_move intent and move the card in Trello

The same as previous one task, but this time it's a bit more difficult, because we need to train our NLU a new intent and move the cards on trello. Please update the above code.

### Scenario 3

We need to setup the variables needed to connect to AVS in the first place. Fill out the variables below.

In [ ]:
PRODUCT_ID = ""
CLIENT_ID = ""
CLIENT_SECRET = ""
WEB_PORT = 9000

The last variable is the refresh token. It is refreshed at each tick, so we need to get it a new one now. To do so we can use the folowing code.

In [ ]:
# source: https://github.com/ewenchou/alexa-client
import cherrypy
import json
import requests
import urllib
import uuid

class Start(object):
    def index(self):
        sd = json.dumps({
            "alexa:all": {
                "productID": PRODUCT_ID,
                "productInstanceAttributes": {
                    "deviceSerialNumber": uuid.getnode()
                }
            }
        })
        url = "https://www.amazon.com/ap/oa"
        callback = "http://localhost:9000/authresponse"#cherrypy.url() + "callback"
        payload = {
            "client_id": CLIENT_ID,
            "scope": "alexa:all",
            "scope_data": sd,
            "response_type": "code",
            "redirect_uri": callback
        }
        req = requests.Request('GET', url, params=payload)
        p = req.prepare()
        raise cherrypy.HTTPRedirect(p.url)

    def authresponse(self, var=None, **params):
        code = urllib.parse.quote(cherrypy.request.params['code'])
        callback = cherrypy.url()
        payload = {
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "code": code,
            "grant_type": "authorization_code",
            "redirect_uri": callback
        }
        url = "https://api.amazon.com/auth/o2/token"
        r = requests.post(url, data=payload)
        resp = r.json()
        return "Refresh token:<br>{}".format(
            resp['refresh_token'])

    index.exposed = True
    authresponse.exposed = True

cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': WEB_PORT})
print('Open http://localhost:9000 to login in amazon alexa service')
cherrypy.quickstart(Start())

Now copy-paste the token below.

In [ ]:
REFRESH_TOKEN=""

##### Text to speech

Now it's time to imitate our voice. Let's use Google Text to Speech API.

In [3]:
from gtts import gTTS

tts = gTTS(text='Alexa, what is the current time in Poland?', lang='en-us', slow=False)
tts.save("hello.mp3")

from pydub import AudioSegment

sound = AudioSegment.from_mp3("hello.mp3")
sound.export("hello.wav", format="wav")

<_io.BufferedRandom name='hello.wav'>

In [4]:
from IPython.display import Audio
sound_file = 'hello.mp3'

Audio(url=sound_file, autoplay=False)

#### Connect to AVS

In [33]:
from avs_client import AlexaVoiceServiceClient

alexa_client = AlexaVoiceServiceClient(
    client_id=CLIENT_ID,
    secret=CLIENT_SECRET,
    refresh_token=REFRESH_TOKEN,
)
alexa_client.connect()

with open('./hello.wav', 'rb') as f:
    alexa_response_audio = alexa_client.send_audio_file(f)
with open('./output2.wav', 'wb') as f:
    f.write(alexa_response_audio)

### Where to go next?

- use generative models [7],
- build an Alexa skill,
- try Stanford library for sentiment analysis [8],
- register for more workshops on bots: workshops.codete.com


##### Glossary

**Intent** - user's goal (intention)

**Sentiment Analysis** - is about the emotions of the user

**Natural Lanaguage Understanding** - is a part of NLP that focus on the more complex part of NLP which is the understanding of sentences

##### References:

[1] **Designing Bots**, Amir Shevat, O’Reilly 2017

[2] https://api.slack.com/methods

[3] https://res.codete.com/sentiment_analysis.pdf

[4] https://github.com/sarumont/py-trello

[5] http://nlu.rasa.ai/

[6] https://spacy.io/

[7] http://cbl.eng.cam.ac.uk/pub/Intranet/MLG/ReadingGroup/Deep_Gen_Janz_Requeima.pdf

[8] https://nlp.stanford.edu/sentiment/

##### Contact

You can contact me by email: karol@codete.com or meet me in Berlin or Kraków at our Codete offices. You can also add me at LinkedIn: https://www.linkedin.com/in/karolprzystalski/